In [1]:
import requests
import pandas as pd
import numpy as np

### Set Up Mongo Connection

import urllib.parse
from pymongo import MongoClient

username = urllib.parse.quote_plus('w210_db_user')
password = urllib.parse.quote_plus('q1w2e3r4$')
conn = MongoClient('mongodb://%s:%s@198.11.212.212:27017/w210_db' % (username, password))
db = conn.w210_db

pd.set_option('display.max_columns', 500)

### Master List of ICOs

Get the master list of 2500+ ICOs to match.

In [2]:
result = db['icotracker_ico'].find({})

data = []
for i,row in enumerate(result):
    data.append(row)
    
master = pd.DataFrame(data)
master['name'] = master.apply(lambda x: str(x['name']).lower(),axis=1)
master.head()

,_id,base,bonus_details,cluster_pca,company_url,description,escrow_class,escrow_href,ico_dates,ico_sold_status,name,owner_linkedin_profile,owner_name,social,status,tracker_url,whitepaper
0,5b1c8cf448d6215496c4d2c0,Ethereum,Start Bonus: 35% - Raised: 0,NaN,https://patron-ico.io/,A decentralized platform which shares economie...,"[fa, fa-close]",NaN,09/05/2018 00:00:00 UTC - 09/06/2018,"ICO: 240,000,000 PAT of 400,000,000",patron,https://www.linkedin.com/in/atsushi-hisatsumi/,Atsushi Hisatsumi,"{'Telegram': 'https://t.me/patronicoen', 'Face...",no_reddit,https://icotracker.net/project/patron,https://patron-ico.io/doc/patron-wp-en-01-2018...
1,5b1c8cf448d6215496c4d2c1,Ethereum,Start Bonus: 30% - Raised: 0,NaN,http://crypton.vc/,A Blockchain-based cryptocurrency venture capi...,"[fa, fa-close]",NaN,29/03/2018 00:00:00 UTC - 09/06/2018,"ICO: 50,400,000 CRN of 72,000,000",crypton vc,https://www.linkedin.com/in/jaceknowak2/,Jacek Nowak,"{'Github': 'https://github.com/CryptonVC', 'Te...",no_reddit,https://icotracker.net/project/crypton_vc,http://crypton.vc/assets/files/white-paper.pdf
2,5b1c8cf548d6215496c4d2c2,Ethereum,Start Bonus: 25% - Raised: 0,NaN,https://www.sosrcoin.io/en/index.html,A cryptocurrency focusing on solving waste pro...,"[fa, fa-close]",NaN,10/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 45,000,000 SOSR of 50,000,000",sosr,https://icotracker.net/www.linkedin.com/in/cha...,Zhen Lin,{'Telegram': 'https://t.me/sosrcoin'},no_reddit,https://icotracker.net/project/sosr,https://drive.google.com/file/d/1jHYIt6TZm3ZTB...
3,5b1c8ef048d6215496c4d2c4,Ethereum,Start Bonus: 50% - Raised: 0,NaN,https://dynatiq.com/,A Blockchain-based marketplace for domains and...,"[fa, fa-close]",NaN,25/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 10,000,000 DTQ of 11,000,000",dynatiq,https://www.linkedin.com/in/abdaluzza/,Abd al Uzza,"{'Blog': 'https://medium.com/@dynatiq', 'Teleg...",no_reddit,https://icotracker.net/project/dynatiq,https://dynatiq.com/whitepaper.pdf
4,5b1c8ef048d6215496c4d2c5,Waves,Start Bonus: 50% - Raised: 0,NaN,https://3ccoin.io/en/,A Blockchain-based option to invest in the pro...,"[fa, fa-close]",NaN,10/05/2018 00:00:00 UTC - 10/06/2018,ICO: ~ 3cCoin ~,3ccoin,https://www.facebook.com/asb-service,Alexey Butorin,{'Youtube': 'https://www.youtube.com/channel/U...,no_reddit,https://icotracker.net/project/3ccoin,https://3ccoin.io/download/en/3cCoin_crowdsale...


In [4]:
# Manipulate some columns

import datetime
def format_ico_dates(dates_text):
    dates = []
    for txt in dates_text.split(' '):
        try:
            valid_date = datetime.datetime.strptime(txt.strip(),'%d/%m/%Y')
            dates.append(datetime.datetime.strftime(valid_date,'%d/%m/%Y'))
        except ValueError as e:
            # Not a date, fail silently...
            pass
    if len(dates) == 1:
        dates = dates*2
    return ' - '.join(dates)

def ico_phase_ended(start_date):
    if start_date.strip() == '' or len(start_date.strip()) < 10:
        return 0
    elif datetime.datetime.strptime(start_date,'%d/%m/%Y') <= datetime.datetime.today():
        return 1
    else:
        return 0

master['key_dates'] = master.apply(lambda x: format_ico_dates(str(x['ico_dates'])),axis=1)
master['ico_phase_ended'] = master.apply(lambda x: ico_phase_ended(x['key_dates'].split(' - ')[-1].strip()),axis=1)
master.head()

,_id,base,bonus_details,cluster_pca,company_url,description,escrow_class,escrow_href,ico_dates,ico_sold_status,name,owner_linkedin_profile,owner_name,social,status,tracker_url,whitepaper,key_dates,ico_phase_ended
0,5b1c8cf448d6215496c4d2c0,Ethereum,Start Bonus: 35% - Raised: 0,NaN,https://patron-ico.io/,A decentralized platform which shares economie...,"[fa, fa-close]",NaN,09/05/2018 00:00:00 UTC - 09/06/2018,"ICO: 240,000,000 PAT of 400,000,000",patron,https://www.linkedin.com/in/atsushi-hisatsumi/,Atsushi Hisatsumi,"{'Telegram': 'https://t.me/patronicoen', 'Face...",no_reddit,https://icotracker.net/project/patron,https://patron-ico.io/doc/patron-wp-en-01-2018...,09/05/2018 - 09/06/2018,1
1,5b1c8cf448d6215496c4d2c1,Ethereum,Start Bonus: 30% - Raised: 0,NaN,http://crypton.vc/,A Blockchain-based cryptocurrency venture capi...,"[fa, fa-close]",NaN,29/03/2018 00:00:00 UTC - 09/06/2018,"ICO: 50,400,000 CRN of 72,000,000",crypton vc,https://www.linkedin.com/in/jaceknowak2/,Jacek Nowak,"{'Github': 'https://github.com/CryptonVC', 'Te...",no_reddit,https://icotracker.net/project/crypton_vc,http://crypton.vc/assets/files/white-paper.pdf,29/03/2018 - 09/06/2018,1
2,5b1c8cf548d6215496c4d2c2,Ethereum,Start Bonus: 25% - Raised: 0,NaN,https://www.sosrcoin.io/en/index.html,A cryptocurrency focusing on solving waste pro...,"[fa, fa-close]",NaN,10/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 45,000,000 SOSR of 50,000,000",sosr,https://icotracker.net/www.linkedin.com/in/cha...,Zhen Lin,{'Telegram': 'https://t.me/sosrcoin'},no_reddit,https://icotracker.net/project/sosr,https://drive.google.com/file/d/1jHYIt6TZm3ZTB...,10/04/2018 - 09/06/2018,1
3,5b1c8ef048d6215496c4d2c4,Ethereum,Start Bonus: 50% - Raised: 0,NaN,https://dynatiq.com/,A Blockchain-based marketplace for domains and...,"[fa, fa-close]",NaN,25/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 10,000,000 DTQ of 11,000,000",dynatiq,https://www.linkedin.com/in/abdaluzza/,Abd al Uzza,"{'Blog': 'https://medium.com/@dynatiq', 'Teleg...",no_reddit,https://icotracker.net/project/dynatiq,https://dynatiq.com/whitepaper.pdf,25/04/2018 - 09/06/2018,1
4,5b1c8ef048d6215496c4d2c5,Waves,Start Bonus: 50% - Raised: 0,NaN,https://3ccoin.io/en/,A Blockchain-based option to invest in the pro...,"[fa, fa-close]",NaN,10/05/2018 00:00:00 UTC - 10/06/2018,ICO: ~ 3cCoin ~,3ccoin,https://www.facebook.com/asb-service,Alexey Butorin,{'Youtube': 'https://www.youtube.com/channel/U...,no_reddit,https://icotracker.net/project/3ccoin,https://3ccoin.io/download/en/3cCoin_crowdsale...,10/05/2018 - 10/06/2018,1


### Source Various Exchange Datasets

In [5]:
# GET request to CryptoCompare
response = requests.get("https://www.cryptocompare.com/api/data/coinlist/")
# Result comes back as a flat dict, transform to a list of dicts
data = []
for k,v in response.json()['Data'].items():
    data.append(v)
# Visualize as a dataframe 
df2 = pd.DataFrame(data)
df2.head()

,Algorithm,CoinName,FullName,FullyPremined,Id,ImageUrl,Name,PreMinedValue,ProofType,SortOrder,Sponsored,Symbol,TotalCoinSupply,TotalCoinsFreeFloat,Url
0,Scrypt,Dogecoin,Dogecoin (DOGE),0,4432,/media/19684/doge.png,DOGE,N/A,PoW,8,False,DOGE,N/A,N/A,/coins/doge/overview
1,Equihash,ZCash,ZCash (ZEC),0,24854,/media/351360/zec.png,ZEC,N/A,PoW,9,False,ZEC,21000000,N/A,/coins/zec/overview
2,SHA-512,Bitshares,Bitshares (BTS),1,5039,/media/20705/bts.png,BTS,N/A,PoS,10,False,BTS,2511953117,N/A,/coins/bts/overview
3,X11,CraigsCoin,CraigsCoin (CRAIG),0,4425,/media/20022/craig.png,CRAIG,N/A,PoS,15,False,CRAIG,30000000,N/A,/coins/craig/overview
4,SHA256,PayCoin,PayCoin (XPY),0,5030,/media/20076/xpy_1.png,XPY,N/A,PoS,17,False,XPY,12500000,N/A,/coins/xpy/overview


In [6]:
# GET request to CryptoWatch
response = requests.get("https://api.cryptowat.ch/assets")
data = response.json()['result']
df3 = pd.DataFrame(data)
df3.head()

,fiat,id,name,route,symbol
0,False,404,FirstBlood,https://api.cryptowat.ch/assets/1st,1st
1,False,201,ICON,https://api.cryptowat.ch/assets/ICX,ICX
2,False,517,The Abyss,https://api.cryptowat.ch/assets/abyss,abyss
3,False,502,Alphacat,https://api.cryptowat.ch/assets/acat,acat
4,False,484,Achain,https://api.cryptowat.ch/assets/act,act


In [7]:
deadcoin = db["deadcoin_ico"]
result = deadcoin.find({})

data = []
for i,row in enumerate(result):
    data.append(row)
df1 = pd.DataFrame(data)

In [11]:
ico_prices = db["icostats_ico_price"]
result = deadcoin.find({})

data = []
for i,row in enumerate(result):
    data.append(row)
df4 = pd.DataFrame(data)
df4.head()

In [22]:
# CoinMarketCap

from bs4 import BeautifulSoup
response = requests.get("https://coinmarketcap.com/all/views/all/")
soup = BeautifulSoup(response.text)
names = [x.text for x in soup.find_all("a", class_="currency-name-container link-secondary")]
df5 = pd.DataFrame(data=names,columns=['name'])

### Merge Function

Define a resusable merge function

In [29]:
def merge_dataset(base,new,indicator,joinOn='name'):
    new[joinOn] = new[joinOn].apply(lambda x: str(x).lower())
    merged = pd.merge(base,new,left_on='name',right_on=joinOn,how='left',indicator=True)
    merged['ind_'+indicator.lower()] = merged['_merge'].apply(lambda x: 0 if x == 'left_only' else 1)
    merged = merged.drop(columns='_merge')
    
    return merged

# Lowercase names
datasets = [
    {'name': 'CoinMarketCap', 'data': df5, 'column': 'name'},
    {'name': 'CryptoCompare', 'data': df2, 'column': 'CoinName'},
    {'name': 'CryptoWatch', 'data': df3, 'column': 'name'},
    {'name': 'RavneetICOPrice', 'data': df4, 'column': 'name'},
    {'name': 'DeadICO','data': df1, 'column': 'name'}
]
df = master
for i,d in enumerate(datasets):
    print('Joining to %s on %s...' % (d['name'],d['column']))
    df = merge_dataset(base=df,new=d['data'],indicator=d['name'].lower(),joinOn=d['column'])

Joining to CoinMarketCap on name...
Joining to CryptoCompare on CoinName...
Joining to CryptoWatch on name...
Joining to RavneetICOPrice on name...
Joining to DeadICO on name...


In [30]:
df.head()

,_id_x,base,bonus_details,cluster_pca,company_url,description,escrow_class,escrow_href,ico_dates,ico_sold_status,name,owner_linkedin_profile,owner_name,social,status_x,tracker_url,whitepaper,key_dates,ico_phase_ended,ind_coinmarketcap,Algorithm,CoinName,FullName,FullyPremined,Id,ImageUrl,Name,PreMinedValue,ProofType,SortOrder,Sponsored,Symbol,TotalCoinSupply,TotalCoinsFreeFloat,Url,ind_cryptocompare,fiat,id,route,symbol,ind_cryptowatch,_id_y,comment_x,rating_x,status_y,url_x,ind_ravneeticoprice,_id,comment_y,rating_y,status,url_y,ind_deadico
0,5b1c8cf448d6215496c4d2c0,Ethereum,Start Bonus: 35% - Raised: 0,NaN,https://patron-ico.io/,A decentralized platform which shares economie...,"[fa, fa-close]",NaN,09/05/2018 00:00:00 UTC - 09/06/2018,"ICO: 240,000,000 PAT of 400,000,000",patron,https://www.linkedin.com/in/atsushi-hisatsumi/,Atsushi Hisatsumi,"{'Telegram': 'https://t.me/patronicoen', 'Face...",no_reddit,https://icotracker.net/project/patron,https://patron-ico.io/doc/patron-wp-en-01-2018...,09/05/2018 - 09/06/2018,1,1,N/A,patron,PATRON (PAT),0,862146,/media/30002231/pat.png,PAT,N/A,N/A,2629,False,PAT,400000000,N/A,/coins/pat/overview,1,False,548.0,https://api.cryptowat.ch/assets/pat,pat,1,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0
1,5b1c8cf448d6215496c4d2c1,Ethereum,Start Bonus: 30% - Raised: 0,NaN,http://crypton.vc/,A Blockchain-based cryptocurrency venture capi...,"[fa, fa-close]",NaN,29/03/2018 00:00:00 UTC - 09/06/2018,"ICO: 50,400,000 CRN of 72,000,000",crypton vc,https://www.linkedin.com/in/jaceknowak2/,Jacek Nowak,"{'Github': 'https://github.com/CryptonVC', 'Te...",no_reddit,https://icotracker.net/project/crypton_vc,http://crypton.vc/assets/files/white-paper.pdf,29/03/2018 - 09/06/2018,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0
2,5b1c8cf548d6215496c4d2c2,Ethereum,Start Bonus: 25% - Raised: 0,NaN,https://www.sosrcoin.io/en/index.html,A cryptocurrency focusing on solving waste pro...,"[fa, fa-close]",NaN,10/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 45,000,000 SOSR of 50,000,000",sosr,https://icotracker.net/www.linkedin.com/in/cha...,Zhen Lin,{'Telegram': 'https://t.me/sosrcoin'},no_reddit,https://icotracker.net/project/sosr,https://drive.google.com/file/d/1jHYIt6TZm3ZTB...,10/04/2018 - 09/06/2018,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0
3,5b1c8ef048d6215496c4d2c4,Ethereum,Start Bonus: 50% - Raised: 0,NaN,https://dynatiq.com/,A Blockchain-based marketplace for domains and...,"[fa, fa-close]",NaN,25/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 10,000,000 DTQ of 11,000,000",dynatiq,https://www.linkedin.com/in/abdaluzza/,Abd al Uzza,"{'Blog': 'https://medium.com/@dynatiq', 'Teleg...",no_reddit,https://icotracker.net/project/dynatiq,https://dynatiq.com/whitepaper.pdf,25/04/2018 - 09/06/2018,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0
4,5b1c8ef048d6215496c4d2c5,Waves,Start Bonus: 50% - Raised: 0,NaN,https://3ccoin.io/en/,A Blockchain-based option to invest in the pro...,"[fa, fa-close]",NaN,10/05/2018 00:00:00 UTC - 10/06/2018,ICO: ~ 3cCoin ~,3ccoin,https://www.facebook.com/asb-service,Alexey Butorin,{'Youtube': 'https://www.youtube.com/channel/U...,no_reddit,https://icotracker.net/project/3ccoin,https://3ccoin.io/download/en/3cCoin_crowdsale...,10/05/2018 - 10/06/2018,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0


In [31]:
flags = ['ico_phase_ended'] + ['ind_'+x['name'].lower() for x in datasets]
df.groupby(flags).aggregate('count')[['name']]

name
ico_phase_ended ind_coinmarketcap ind_cryptocompare ind_cryptowatch ind_ravneeticoprice ind_deadico      
0               0                 0                 0               0                   0             269
                                                                    1                   1               8
                                  1                 0               0                   0              13
                1                 1                 1               0                   0               1
1               0                 0                 0               0                   0            1851
                                                                    1                   1              13
                                                    1               0                   0               1
                                  1                 0               0                   0             200
                                                                    1                   1               2
                                                    1               0                   0               1
                1                 0                 0               0                   0              34
                                                    1               0                   0              19
                                  1                 0               0                   0             136
                                                                    1                   1               6
                                                    1               0                   0              96

### Notes

+ Only 21 ICOs match dead coin
+ **CryptoWatch** only yields an incremental 20 quotes
+ **CryptoCompare** matches roughly 25% of ICOs
+ **CoinMarketCap** matches roughly 20% of ICOs, but with a significant overlap to CryptoCompare

In [34]:
cols = ['name','symbol'] + flags
data = df[cols].to_dict(orient='records')
for row in data:
    db.sp_overlap_analysis.insert_one(row)

In [36]:
result = db["sp_overlap_analysis"].find({})

data = []
for i,row in enumerate(result):
    data.append(row)
df1 = pd.DataFrame(data)
df1.head()

,_id,ico_phase_ended,ind_coinmarketcap,ind_cryptocompare,ind_cryptowatch,ind_deadico,ind_ravneeticoprice,name,symbol
0,5b40568a57c41002aa12a53a,1,1,1,1,0,0,patron,pat
1,5b40568a57c41002aa12a53b,1,0,0,0,0,0,crypton vc,NaN
2,5b40568a57c41002aa12a53c,1,0,0,0,0,0,sosr,NaN
3,5b40568a57c41002aa12a53d,1,0,0,0,0,0,dynatiq,NaN
4,5b40568a57c41002aa12a53e,1,0,0,0,0,0,3ccoin,NaN
